# Create geojson file from list of PUDO coords
See /home/test/Documents/PROJECTS/VFH/iPython_Notebooks/dv_pudo_profiles.ipynb for more info.  

ref: https://geoffboeing.com/2015/10/exporting-python-data-geojson/  
Huom: `with open(output_filename, 'wb') as output_file:` should be `'w'` not `'wb'`  

**Resources**  
geoJSON minifier: https://jsonformatter.org/json-minify  

In [1]:
import sys
sys.executable

'/home/nangini/anaconda3/bin/python'

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import pandas.io.sql as pandasql
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.ticker as ticker

from IPython.display import HTML, display
import datetime
import matplotlib.dates as mdates
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import os

from datetime import datetime

import json

# Functions

In [3]:
# https://geoffboeing.com/2015/10/exporting-python-data-geojson/
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = int(row[prop])
        geojson['features'].append(feature)
    return geojson

# GeoJSON properties for file

In [4]:
# Define the properties of the PUDO geojson file
prop = ['type','counts', 'overlap']

# Variables

In [5]:
ward=1
this_day = "Monday"
this_tod = "pmPeak"

In [6]:
if ward == 1:
    test_id1 = 20232410   # 13454935
    test_id2 = 13453946
elif ward == 10:
    test_id1 = 13467106 # Adelaide and John
    test_id2 = 13466303

# Read PUDO data from csv file

In [7]:
# fname = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/all_pudo_mon.csv'
fname = '~/Documents/PROJECTS/BDITTO/VFH/DATA_STORY/DATA/from_pgAdmin/all_pudo_mon.csv'

df = pd.read_csv(fname)

print(df.shape)
df.head(5)

(494842, 14)


,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
0,2018-09-03 00:00:00,2018-09-03 00:34:01,18.0,10.0,13447475,13467654,Monday,nightII,0101000020E61000005BE2AF6670DA53C01D22DBEA93E2...,0101000020E610000013E6CE309BD953C0938FE7777DD2...,43.770139,-79.413110,43.644454,-79.400097
1,2018-09-03 00:00:00,2018-09-03 00:10:54,10.0,13.0,13467763,13464030,Monday,nightII,0101000020E610000051E519D30AD953C0C23B89A05BD2...,0101000020E610000079E6DB3E68D853C00F3AF08356D5...,43.643421,-79.391286,43.666703,-79.381363
2,2018-09-03 00:00:00,2018-09-03 00:12:34,13.0,10.0,13464551,13467763,Monday,nightII,0101000020E6100000A342F6F3C6D753C0E8EE4CFEEDD4...,0101000020E610000051E519D30AD953C0C23B89A05BD2...,43.663513,-79.371518,43.643421,-79.391286
3,2018-09-03 00:00:00,2018-09-03 00:24:18,13.0,3.0,13465140,13470547,Monday,nightII,0101000020E6100000DF5023ED39D853C020ED2E3976D4...,0101000020E610000004410BFFE5E153C041B447898CCC...,43.659858,-79.378536,43.598039,-79.529663
4,2018-09-03 00:00:00,2018-09-03 00:18:21,8.0,13.0,14044860,14673305,Monday,nightII,0101000020E6100000A37FC1EFB8DC53C070C432B4FCDB...,0101000020E61000008C84E11C68D853C062D98093A3D5...,43.718649,-79.448788,43.669054,-79.381355


In [8]:
list(df)

['pickup_datetime',
 'dropoff_datetime',
 'pickup_ward2018',
 'dropoff_ward2018',
 'pickup_gc_intersection_id',
 'dropoff_gc_intersection_id',
 'dow',
 'timewindow',
 'pickup_geom',
 'dropoff_geom',
 'lat_pickup',
 'lon_pickup',
 'lat_dropoff',
 'lon_dropoff']

In [9]:
print(df.loc[(df['pickup_gc_intersection_id']==test_id1) & (df['dow']=='Monday') & (df['timewindow']=='amPeak'),'pickup_gc_intersection_id'].count())
df.loc[(df['pickup_gc_intersection_id']==test_id1) & (df['dow']=='Monday') & (df['timewindow']=='amPeak')]

12


,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff
38950,2018-09-03 07:00:00,2018-09-03 07:08:41,1.0,1.0,20232410,30081632,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E61000002D3BBACD78E353C00F64DF43F9DA...,43.728927,-79.60573,43.710732,-79.554248
88959,2018-09-10 08:00:00,2018-09-10 08:12:39,1.0,1.0,20232410,20232410,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,43.728927,-79.60573,43.728927,-79.605730
97207,2018-09-17 07:00:00,2018-09-17 07:12:16,1.0,1.0,20232410,13458127,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E61000002B190A1CACE453C05F98C323EED9...,43.728927,-79.60573,43.702580,-79.573005
140853,2018-09-24 07:00:00,2018-09-24 07:21:00,1.0,1.0,20232410,30081635,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E6100000786CE01B3EE353C0C5D0B23AAEDA...,43.728927,-79.60573,43.708442,-79.550666
140854,2018-09-24 07:00:00,2018-09-24 07:21:00,1.0,1.0,20232410,30081635,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E6100000786CE01B3EE353C0C5D0B23AAEDA...,43.728927,-79.60573,43.708442,-79.550666
183409,2018-09-03 07:00:00,2018-09-03 07:15:17,1.0,3.0,20232410,30071354,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E6100000A29221642CE253C0679E157ED4CF...,43.728927,-79.60573,43.623672,-79.533959
240279,2018-09-17 08:00:00,2018-09-17 08:47:05,1.0,21.0,20232410,13449905,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E610000043B0776D19D153C0E5BAE7C823E0...,43.728927,-79.60573,43.751092,-79.267177
240582,2018-09-17 08:00:00,2018-09-17 08:22:07,1.0,3.0,20232410,13470591,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E6100000BD3611B95DE153C07D60FC7151CC...,43.728927,-79.60573,43.596236,-79.521345
246059,2018-09-17 07:00:00,2018-09-17 07:58:57,1.0,21.0,20232410,20102020,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E61000004EDEF8C1F6D053C03FE99A8262E0...,43.728927,-79.60573,43.753006,-79.265061
258887,2018-09-24 08:00:00,2018-09-24 08:11:44,1.0,1.0,20232410,13460408,Monday,amPeak,0101000020E610000075DC3A49C4E653C09A9B9F7D4DDD...,0101000020E61000003C54D9CB04E553C07379977E4ED8...,43.728927,-79.60573,43.689895,-79.578418


In [10]:
df.loc[(df['dropoff_gc_intersection_id']==test_id1) & (df['dow']=='Monday') & (df['timewindow']=='amPeak'),'dropoff_gc_intersection_id'].count()

151

In [11]:
print(df.loc[(df['pickup_gc_intersection_id']==test_id1) & 
       (df['dow']=='Monday') & (df['timewindow']=='pmPeak') &
       (df['pickup_ward2018']==10.0),
       'dropoff_gc_intersection_id'].count())

df.loc[(df['pickup_gc_intersection_id']==test_id1) & 
       (df['dow']=='Monday') & (df['timewindow']=='pmPeak') &
       (df['pickup_ward2018']==10.0), ['pickup_datetime','pickup_gc_intersection_id','dropoff_ward2018','dow','timewindow']]

0


,pickup_datetime,pickup_gc_intersection_id,dropoff_ward2018,dow,timewindow


In [12]:
print(df.loc[(df['pickup_gc_intersection_id']==test_id2) & 
       (df['dow']=='Monday') & (df['timewindow']=='pmPeak') &
       (df['pickup_ward2018']==10.0),
       'dropoff_gc_intersection_id'].count())

df.loc[(df['pickup_gc_intersection_id']==test_id2) & 
       (df['dow']=='Monday') & (df['timewindow']=='pmPeak') &
       (df['pickup_ward2018']==10.0), ['pickup_datetime','pickup_gc_intersection_id','dropoff_ward2018','dow','timewindow']]

0


,pickup_datetime,pickup_gc_intersection_id,dropoff_ward2018,dow,timewindow


# Housekeeping

## 1. Round lat, lon to 4 decimal places

In [13]:
n=4 # number of decimals to round to

df['lat_pickup'] = df['lat_pickup'].round(decimals=n)
df['lon_pickup'] = df['lon_pickup'].round(decimals=n)

df['lat_dropoff'] = df['lat_dropoff'].round(decimals=n)
df['lon_dropoff'] = df['lon_dropoff'].round(decimals=n)

In [14]:
np.round(-79.60573035, decimals=n)

-79.6057

In [15]:
np.round(43.7289273, decimals=n)

43.7289

## 2. Filter out rows where neither pickup or dropoff wards are the ward of interest
Make col `keep` = 1 for where pick-up or for drop-off wards are the ward of interest

In [16]:
# Assign a Pickup or Dropoff in column 'type'
df['keep'] = np.nan
df['latitude'] = np.nan
df['longitude'] = np.nan

df.loc[(df['pickup_ward2018']==ward) | (df['dropoff_ward2018']==ward), 'keep'] = 1

In [17]:
# Check
df['keep'].unique()

array([nan,  1.])

In [18]:
# Keep only rows where `keep` is not null
df = df[df['keep'].notnull()]

In [19]:
# Check
df['keep'].unique()

array([1.])

In [20]:
print(df.shape)

(18462, 17)


In [21]:
df.head(5)

,pickup_datetime,dropoff_datetime,pickup_ward2018,dropoff_ward2018,pickup_gc_intersection_id,dropoff_gc_intersection_id,dow,timewindow,pickup_geom,dropoff_geom,lat_pickup,lon_pickup,lat_dropoff,lon_dropoff,keep,latitude,longitude
6,2018-09-03 00:00:00,2018-09-03 00:13:26,1.0,1.0,30082386,13460542,Monday,nightII,0101000020E61000002E8E37EAE2E653C01D39868BD2DC...,0101000020E61000001223231E8BE653C012195BAA40D8...,43.7252,-79.6076,43.6895,-79.6022,1.0,NaN,NaN
65,2018-09-03 00:00:00,2018-09-03 00:13:36,6.0,1.0,30109966,13451158,Monday,nightII,0101000020E6100000F094730106DF53C0346577E63DDF...,0101000020E61000002595DD571BE553C04E5E8510A7DF...,43.7441,-79.4847,43.7473,-79.5798,1.0,NaN,NaN
77,2018-09-03 00:00:00,2018-09-03 00:22:29,1.0,9.0,13461025,14047180,Monday,nightII,0101000020E6100000A9ABCBC11DE653C075770AAAE7D7...,0101000020E6100000B94BF948BEDB53C0D4C5FE9626D6...,43.6868,-79.5956,43.6731,-79.4335,1.0,NaN,NaN
92,2018-09-03 01:00:00,2018-09-03 01:04:16,1.0,1.0,13451763,13450652,Monday,nightII,0101000020E6100000716B477BC0E653C0A2CEA29838DF...,0101000020E6100000DF7A0DF462E653C0EC776DDE14E0...,43.7439,-79.6055,43.7506,-79.5998,1.0,NaN,NaN
110,2018-09-03 00:00:00,2018-09-03 00:42:09,25.0,1.0,13441921,14075750,Monday,nightII,0101000020E61000008BC12B98A4CD53C04A9D99C3E2E8...,0101000020E6100000BBD25DD28BE653C029598BA581DC...,43.8194,-79.2132,43.7227,-79.6023,1.0,NaN,NaN


In [22]:
# Test again!! => ok
print(df.loc[(df['pickup_gc_intersection_id']==test_id1) & (df['dow']=='Monday') & (df['timewindow']=='amPeak'),'pickup_gc_intersection_id'].count())
df.loc[(df['dropoff_gc_intersection_id']==test_id1) & (df['dow']=='Monday') & (df['timewindow']=='amPeak'),'dropoff_gc_intersection_id'].count()

12


151

In [23]:
# Test again!! => ok
print(df.loc[(df['pickup_gc_intersection_id']==test_id1) & (df['dow']==this_day) & 
             (df['pickup_ward2018']==ward) &
             (df['timewindow']==this_tod),'pickup_gc_intersection_id'].count())

53


## 3. Mark if row is a pickup or a dropoff
pickup is where pickup ward == ward, dropoff is where dropoff_ward == ward  
Note that a row can contain **both** a pickup and a dropoff !!!  

In [24]:
df['pu'] = np.nan
df['do'] = np.nan

In [25]:
df.loc[df['pickup_ward2018']==ward, 'pu'] = 1

In [26]:
df.loc[df['dropoff_ward2018']==ward, 'do'] = 1

In [27]:
# Check
df.loc[df['dropoff_ward2018']==ward, 
       ['dow','timewindow','pickup_datetime','dropoff_datetime',
        'lat_pickup','lon_pickup','pickup_gc_intersection_id',
        'lat_dropoff','lon_dropoff','dropoff_gc_intersection_id',
        'pickup_ward2018','dropoff_ward2018','pu','do']].head(11)

,dow,timewindow,pickup_datetime,dropoff_datetime,lat_pickup,lon_pickup,pickup_gc_intersection_id,lat_dropoff,lon_dropoff,dropoff_gc_intersection_id,pickup_ward2018,dropoff_ward2018,pu,do
6,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:13:26,43.7252,-79.6076,30082386,43.6895,-79.6022,13460542,1.0,1.0,1.0,1.0
65,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:13:36,43.7441,-79.4847,30109966,43.7473,-79.5798,13451158,6.0,1.0,NaN,1.0
92,Monday,nightII,2018-09-03 01:00:00,2018-09-03 01:04:16,43.7439,-79.6055,13451763,43.7506,-79.5998,13450652,1.0,1.0,1.0,1.0
110,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:42:09,43.8194,-79.2132,13441921,43.7227,-79.6023,14075750,25.0,1.0,NaN,1.0
111,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:08:17,43.7585,-79.5930,30091827,43.7338,-79.5935,13453287,1.0,1.0,1.0,1.0
126,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:03:47,43.7199,-79.5608,13455313,43.7133,-79.5597,13456371,1.0,1.0,1.0,1.0
181,Monday,nightII,2018-09-03 00:00:00,2018-09-03 00:30:08,43.7923,-79.4411,13444614,43.7169,-79.6035,13455879,18.0,1.0,NaN,1.0
277,Monday,nightII,2018-09-03 01:00:00,2018-09-03 01:04:27,43.6908,-79.5974,13460293,43.6783,-79.5850,13462531,1.0,1.0,1.0,1.0
389,Monday,outside_windows,2018-09-03 02:00:00,2018-09-03 02:07:40,43.7297,-79.5643,13453937,43.7325,-79.6010,13453509,1.0,1.0,1.0,1.0
392,Monday,outside_windows,2018-09-03 02:00:00,2018-09-03 02:07:25,43.7087,-79.4781,13456952,43.7015,-79.5573,13458251,5.0,1.0,NaN,1.0


## 4a. Aggregate pickups and dropoffs

In [28]:
df_agg_pu = pd.DataFrame()

df_agg_pu = df.groupby(
    ['dow','timewindow','lat_pickup','lon_pickup','pickup_gc_intersection_id']
    ).size().reset_index(name='pcounts')

df_agg_pu = df_agg_pu.rename(columns={'lat_pickup': 'latitude'})
df_agg_pu = df_agg_pu.rename(columns={'lon_pickup': 'longitude'})
df_agg_pu = df_agg_pu.rename(columns={'pickup_gc_intersection_id': 'intersection_id'})

df_agg_pu.head(11)

,dow,timewindow,latitude,longitude,intersection_id,pcounts
0,Monday,amPeak,43.5908,-79.5463,13470718,1
1,Monday,amPeak,43.5924,-79.5331,13470684,2
2,Monday,amPeak,43.5947,-79.5423,13470634,2
3,Monday,amPeak,43.5959,-79.5292,14301519,1
4,Monday,amPeak,43.5966,-79.5303,14302119,1
5,Monday,amPeak,43.5970,-79.5497,13470582,2
6,Monday,amPeak,43.5985,-79.5473,13470539,2
7,Monday,amPeak,43.5995,-79.5235,13470503,1
8,Monday,amPeak,43.6008,-79.5053,13470452,1
9,Monday,amPeak,43.6010,-79.5087,13470444,1


In [29]:
df_agg_pu.loc[(df_agg_pu['intersection_id']==13448583) & 
                 (df_agg_pu['timewindow']=='amPeak')                 
                ]

,dow,timewindow,latitude,longitude,intersection_id,pcounts
958,Monday,amPeak,43.7642,-79.5532,13448583,3


In [30]:
df_agg_pu.loc[(df_agg_pu['intersection_id']==13470718) # & 
                 # (df_agg_pu['timewindow']=='amPeak')                 
                ]

,dow,timewindow,latitude,longitude,intersection_id,pcounts
0,Monday,amPeak,43.5908,-79.5463,13470718,1
2624,Monday,nightI,43.5908,-79.5463,13470718,3


In [31]:
df_agg_pu.loc[(df_agg_pu['intersection_id']==13470192) # & 
                 # (df_agg_pu['timewindow']=='amPeak')                 
                ]

,dow,timewindow,latitude,longitude,intersection_id,pcounts
3930,Monday,nightII,43.6099,-79.5521,13470192,2
4927,Monday,outside_windows,43.6099,-79.5521,13470192,1


In [32]:
df_agg_pu.head(11)

,dow,timewindow,latitude,longitude,intersection_id,pcounts
0,Monday,amPeak,43.5908,-79.5463,13470718,1
1,Monday,amPeak,43.5924,-79.5331,13470684,2
2,Monday,amPeak,43.5947,-79.5423,13470634,2
3,Monday,amPeak,43.5959,-79.5292,14301519,1
4,Monday,amPeak,43.5966,-79.5303,14302119,1
5,Monday,amPeak,43.5970,-79.5497,13470582,2
6,Monday,amPeak,43.5985,-79.5473,13470539,2
7,Monday,amPeak,43.5995,-79.5235,13470503,1
8,Monday,amPeak,43.6008,-79.5053,13470452,1
9,Monday,amPeak,43.6010,-79.5087,13470444,1


In [33]:
df_agg_do = pd.DataFrame()

df_agg_do = df.groupby(
    ['dow','timewindow','lat_dropoff','lon_dropoff','dropoff_gc_intersection_id']
    ).size().reset_index(name='dcounts')

df_agg_do = df_agg_do.rename(columns={'lat_dropoff': 'latitude'})
df_agg_do = df_agg_do.rename(columns={'lon_dropoff': 'longitude'})
df_agg_do = df_agg_do.rename(columns={'dropoff_gc_intersection_id': 'intersection_id'})

df_agg_do.head(11)

,dow,timewindow,latitude,longitude,intersection_id,dcounts
0,Monday,amPeak,43.5908,-79.5463,13470718,2
1,Monday,amPeak,43.5943,-79.5163,20112511,2
2,Monday,amPeak,43.5962,-79.5213,13470591,3
3,Monday,amPeak,43.5966,-79.5131,30099158,1
4,Monday,amPeak,43.5971,-79.5217,13470569,2
5,Monday,amPeak,43.5977,-79.5191,13470552,1
6,Monday,amPeak,43.5982,-79.5170,13470536,8
7,Monday,amPeak,43.5988,-79.5139,13470520,1
8,Monday,amPeak,43.5998,-79.5331,13470496,1
9,Monday,amPeak,43.6011,-79.5161,13470443,1


In [34]:
print(df_agg_pu.shape)
print(df_agg_do.shape)

(7659, 6)
(7355, 6)


## 5. Subset this_tod

### 5a. Separate pick-ups only, and drop-offs only

In [35]:
print(this_tod)
df_sub_pu = df_agg_pu.loc[(df_agg_pu['timewindow']==this_tod)]

print(df_sub_pu.shape)
df_sub_pu.head(11)

pmPeak
(948, 6)


,dow,timewindow,latitude,longitude,intersection_id,pcounts
6711,Monday,pmPeak,43.5943,-79.5163,20112511,1
6712,Monday,pmPeak,43.5962,-79.5213,13470591,1
6713,Monday,pmPeak,43.5970,-79.5225,13470578,1
6714,Monday,pmPeak,43.5974,-79.5202,13470558,1
6715,Monday,pmPeak,43.5982,-79.5170,13470536,3
6716,Monday,pmPeak,43.5988,-79.5139,13470520,1
6717,Monday,pmPeak,43.6008,-79.5053,13470452,1
6718,Monday,pmPeak,43.6009,-79.5171,13470453,1
6719,Monday,pmPeak,43.6015,-79.5019,13470424,1
6720,Monday,pmPeak,43.6084,-79.5587,13470244,4


In [36]:
# Check for duplicate intersection_id
df_sub_pu.loc[df_sub_pu['intersection_id'].duplicated()]

,dow,timewindow,latitude,longitude,intersection_id,pcounts


In [37]:
print(this_tod)
df_sub_do = df_agg_do.loc[(df_agg_pu['timewindow']==this_tod)]

print(df_sub_do.shape)
df_sub_do.head(11)

pmPeak
(644, 6)


,dow,timewindow,latitude,longitude,intersection_id,dcounts
6711,Monday,pmPeak,43.7090,-79.5522,13457057,2
6712,Monday,pmPeak,43.7091,-79.3933,13456722,1
6713,Monday,pmPeak,43.7092,-79.5655,13457060,6
6714,Monday,pmPeak,43.7097,-79.5922,13457008,1
6715,Monday,pmPeak,43.7097,-79.5346,13456916,1
6716,Monday,pmPeak,43.7100,-79.5015,13456810,2
6717,Monday,pmPeak,43.7103,-79.5301,14171759,2
6718,Monday,pmPeak,43.7106,-79.3939,13456452,1
6719,Monday,pmPeak,43.7107,-79.5542,30081632,1
6720,Monday,pmPeak,43.7110,-79.5361,13456716,1


In [38]:
# Check for duplicate intersection_id
df_sub_do.loc[df_sub_do['intersection_id'].duplicated()]

,dow,timewindow,latitude,longitude,intersection_id,dcounts


### 5b. Stack pickups on top of dropoffs

In [39]:
# Add empty columns to allow stacking

df_sub_pu=df_sub_pu.assign(dcounts=0)
df_sub_do=df_sub_do.assign(pcounts=0)

In [40]:
# stack
print(df_sub_pu.shape)
print(df_sub_do.shape)

df_merge=pd.concat([df_sub_pu, df_sub_do],sort=False)

print(df_merge.shape)
df_merge.head(11)

(948, 7)
(644, 7)
(1592, 7)


,dow,timewindow,latitude,longitude,intersection_id,pcounts,dcounts
6711,Monday,pmPeak,43.5943,-79.5163,20112511,1,0
6712,Monday,pmPeak,43.5962,-79.5213,13470591,1,0
6713,Monday,pmPeak,43.5970,-79.5225,13470578,1,0
6714,Monday,pmPeak,43.5974,-79.5202,13470558,1,0
6715,Monday,pmPeak,43.5982,-79.5170,13470536,3,0
6716,Monday,pmPeak,43.5988,-79.5139,13470520,1,0
6717,Monday,pmPeak,43.6008,-79.5053,13470452,1,0
6718,Monday,pmPeak,43.6009,-79.5171,13470453,1,0
6719,Monday,pmPeak,43.6015,-79.5019,13470424,1,0
6720,Monday,pmPeak,43.6084,-79.5587,13470244,4,0


In [41]:
# Check
this_id=13446142
df_merge.loc[df_merge['intersection_id']==this_id]

,dow,timewindow,latitude,longitude,intersection_id,pcounts,dcounts
7644,Monday,pmPeak,43.7798,-79.4156,13446142,3,0
7335,Monday,pmPeak,43.7798,-79.4156,13446142,0,1


### 5c. Group on common `intersection_id`

In [42]:
df_merge_gr=pd.DataFrame()

df_merge_gr=df_merge.groupby('intersection_id')[['latitude','longitude','pcounts','dcounts']].sum().reset_index()

print(df_merge_gr.shape)
df_merge_gr.head(11)

(1306, 5)


,intersection_id,latitude,longitude,pcounts,dcounts
0,13441844,43.8223,-79.2757,0,1
1,13442730,43.8072,-79.2704,0,1
2,13443027,43.8030,-79.2144,0,1
3,13443191,43.8014,-79.2315,1,0
4,13443304,43.8013,-79.3019,0,1
5,13443680,43.7995,-79.4089,0,1
6,13443868,43.7971,-79.3519,1,0
7,13443998,43.7957,-79.3281,1,0
8,13444159,43.7947,-79.3552,0,1
9,13444186,43.7944,-79.3474,0,1


In [43]:
# Check
this_id=13446142
df_merge_gr.loc[df_merge_gr['intersection_id']==this_id]

,intersection_id,latitude,longitude,pcounts,dcounts
26,13446142,87.5596,-158.8312,3,1


## 6. Make the dataframes for each layer!

In [44]:
df_pu_layer = df_merge_gr.loc[(df_merge_gr['pcounts']>0) & (df_merge_gr['dcounts']==0)]

df_do_layer = df_merge_gr.loc[(df_merge_gr['pcounts']==0) & (df_merge_gr['dcounts']>0)]

df_pudo_layer = df_merge_gr.loc[(df_merge_gr['pcounts']>0) & (df_merge_gr['dcounts']>0)]

In [45]:
print(df_pu_layer.shape)
print(df_do_layer.shape)
print(df_pudo_layer.shape)

(662, 5)
(358, 5)
(286, 5)


In [46]:
df=

SyntaxError: invalid syntax (<ipython-input-46-02c2a1c0215a>, line 1)

# Save to geojson

In [47]:
print('w' +repr(ward), this_day, this_tod)

w1 Monday pmPeak


In [63]:
df_do_layer.head(5)

,intersection_id,latitude,longitude,pcounts,dcounts
0,13441844,43.8223,-79.2757,0,1
1,13442730,43.8072,-79.2704,0,1
2,13443027,43.8030,-79.2144,0,1
4,13443304,43.8013,-79.3019,0,1
5,13443680,43.7995,-79.4089,0,1


In [65]:
df_do_layer.sort_values('dcounts',ascending=False)

,intersection_id,latitude,longitude,pcounts,dcounts
495,13453618,43.7317,-79.5764,0,9
87,13448814,43.7626,-79.5561,0,9
761,13456027,43.7158,-79.5898,0,8
360,13452484,43.7391,-79.5928,0,8
753,13455939,43.7165,-79.6087,0,6
436,13453198,43.7337,-79.5382,0,4
620,13454784,43.7234,-79.5724,0,4
28,13446229,43.7766,-79.2542,0,4
138,13449657,43.7568,-79.5681,0,3
594,13454479,43.7254,-79.5576,0,3


In [75]:
df_do_layer.loc[(df_do_layer['dcounts']>=c)]

,intersection_id,latitude,longitude,pcounts,dcounts
87,13448814,43.7626,-79.5561,0,9
495,13453618,43.7317,-79.5764,0,9


## Apply cut-off filter
Only counts > `c` will be saved.  

In [71]:
c = 9 # cut-off

In [72]:
geojson_pu_layer = df_to_geojson(df_pu_layer.loc[(df_pu_layer['pcounts']>=c)], ['pcounts'])

In [73]:
geojson_do_layer = df_to_geojson(df_do_layer.loc[(df_do_layer['dcounts']>=c)], ['dcounts'])

In [74]:
geojson_pudo_layer = df_to_geojson(df_pudo_layer.loc[(df_pudo_layer['pcounts']>=c) &
                                                     (df_pudo_layer['dcounts']>=c)
                                                    ], ['pcounts','dcounts'])

# Write geojson to file

## Aggregated points

In [76]:
print('w' +repr(ward), this_day, this_tod)

w1 Monday pmPeak


In [77]:
# mydir = '../../../Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/'
mydir = '/home/nangini/Documents/PROJECTS/BDITTO/VFH/DATA_STORY/DATA/INDIVIDUAL_GEOJSONS/'


output_filename = mydir + 'w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_pu_cutoff.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_pu_layer, output_file, indent=2) 

In [78]:
output_filename = mydir + 'w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_do_cutoff.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_do_layer, output_file, indent=2) 

In [79]:
output_filename = mydir + 'w' + repr(ward) + '_092018_' + this_day + '_' + this_tod + '_pudo_cutoff.geojson'

with open(output_filename, 'w') as output_file:
    output_file.write('')
    json.dump(geojson_pudo_layer, output_file, indent=2) 

`
$ ls -lh ~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/INDIVIDUAL_GEOJSONS/
total 216K
-rw-r--r-- 1 test test 109K Dec  9 11:23 w1_092018_Monday_pmPeak_do_agg.geojson
-rw-r--r-- 1 test test 104K Dec  9 11:23 w1_092018_Monday_pmPeak_pu_agg.geojson
`


`
$ ls -lh src/geojson/
-rw-r--r-- 1 test test 328K Dec  4 15:46 w1_092018_Monday_amPeak_agg.geojson
-rw-r--r-- 1 test test 753K Dec  4 13:51 w1_092018_Monday_amPeak.geojson
`
